In [1]:
from utils import *
from matplotlib.lines import Line2D
from scipy.stats import ttest_rel

import warnings
# warnings.filterwarnings('always')


In [2]:
dic_region = {
    "Northern Africa and Western Asia": "N. Africa W. Asia",
    "Sub-Saharan Africa": "Sub-Saharan Africa",
    "Central Asia and Russian Federation": "C. Asia",
    "Eastern Asia": "E. Asia",
    "Southern Asia": "S. Asia",
    "Southeastern Asia": "S. Asia",
    "Northern America": "N. America",
    "Latin America and the Caribbean": "Latin America",
    "Western Europe": "W. Europe",
    "Eastern and South-Eastern Europe": "E. Europe",
    "Oceania and Australia": "Oceania",
}
gdf_countries = gdf_world.reset_index(drop=True).reset_index().rename(columns={"index": "idx"})
gdf_countries["regi_short"] = gdf_countries["regi_pnas"].map(dic_region)
country_to_region = gdf_countries.set_index("name_long")["regi_short"].to_dict()

In [3]:
regions = ['C. Asia', 'E. Asia', 'S. Asia', 'N. Africa W. Asia', 'Sub-Saharan Africa', 'N. America', 'Latin America', 'Oceania', 'E. Europe', 'W. Europe']
regions_to_color = dict(zip(regions, plt.get_cmap("tab20")(np.linspace(0, 1, len(regions)))))

(path_data / "ring/ttest").mkdir(exist_ok=True)

In [4]:
def load_df_diff_matched(ring_radius):
    df_matched_m_lst = []
    df_diff_m_lst = []
    for year in range(2002, 2024):
        df_matched_m_lst.append(pd.read_csv(path_data / f"ring/df_matched_ring_{ring_radius}km_{year}.csv", index_col=0))
        df_diff_m_lst.append(pd.read_csv(path_data / f"ring/df_diff_ring_{ring_radius}km_{year}.csv", index_col=0))
    df_diff_m_all = pd.concat(df_diff_m_lst)
    df_matched_m_all = pd.concat(df_matched_m_lst)
    return df_matched_m_all, df_diff_m_all

In [ ]:
ring_radius = 25
df_matched_m_all, df_diff_m_all = load_df_diff_matched(ring_radius)
df_diff_m_all.loc[:, "country"] = df_matched_m_all["country"]

# ttest function

In [ ]:
def ttest_1luc(df_matched_m_all, df_diff_m_all, luc="crop"):
    df_match_m_1luc = df_matched_m_all.reset_index(drop=True)
    df_match_m_1luc["region"] = df_match_m_1luc["country"].map(country_to_region)

    df_diff_m_1luc = df_diff_m_all.reset_index(drop=True)
    df_diff_m_1luc["region"] = df_diff_m_1luc["country"].map(country_to_region)

    df_npp_diff_country = pd.read_csv(path_data / f"npp/{luc}_npp_diff.csv").rename(columns={"name_long": "country"})
    df_npp_diff_country = df_npp_diff_country.melt(id_vars="country", var_name="year", value_name=f"{luc}_npp_diff_country")\
        .assign(year=lambda _df: _df["year"].astype(int))
    df_match_m_1luc = df_match_m_1luc.merge(df_npp_diff_country, on=["country", "year"], how="left")
    df_match_m_1luc.loc[:, f"{luc}_npp_change_n"] = df_match_m_1luc[[f"{luc}_npp_change_n", f"{luc}_npp_diff_country"]].bfill(axis=1).iloc[:, 0]
        
    ttest_1luc_data = []
    for (year, country), df_gp in list(df_match_m_1luc.groupby(["year", "country"])):
        df_gp = df_gp[[f"{luc}_change", f"{luc}_change_n"]].copy().dropna()
        if df_gp.shape[0] < 5:
            t_stat, t_p, diff_mean, diff_std = np.nan, np.nan, np.nan, np.nan
        else:
            t_stat, t_p = ttest_rel(df_gp[f"{luc}_change"].values, df_gp[f"{luc}_change_n"].values)
            diff_mean = np.mean(df_gp[f"{luc}_change"].values - df_gp[f"{luc}_change_n"].values)
            diff_std = np.std(df_gp[f"{luc}_change"].values - df_gp[f"{luc}_change_n"].values)
        ttest_1luc_data.append([
            year, country, t_stat, t_p, diff_mean, diff_std
        ])
    df_1luc_change_tt_test = pd.DataFrame(ttest_1luc_data, columns=[
        "year", "country", "t_stat", "t_p", "diff_mean", "diff_std"
    ])#.sort_values("diff_mean")
    
    ttest_1luc_data = []
    for (year, country), df_gp in list(df_match_m_1luc.groupby(["year", "country"])):
        with warnings.catch_warnings(record=True) as w:
            df_gp = df_gp[[f"{luc}_npp_change", f"{luc}_npp_change_n"]].copy().dropna()
            if df_gp.shape[0] < 5:
                t_stat, t_p, diff_mean, diff_std = np.nan, np.nan, np.nan, np.nan
            else:
                t_stat, t_p = ttest_rel(df_gp[f"{luc}_npp_change"].values, df_gp[f"{luc}_npp_change_n"].values)
                diff_mean = np.mean(df_gp[f"{luc}_npp_change"].values - df_gp[f"{luc}_npp_change_n"].values)
                diff_std = np.std(df_gp[f"{luc}_npp_change"].values - df_gp[f"{luc}_npp_change_n"].values)
                if w:
                    for warning in w:
                        print(year, country)
            ttest_1luc_data.append([
                year, country, t_stat, t_p, diff_mean, diff_std])
        # break
    df_1luc_npp_change_tt_test = pd.DataFrame(ttest_1luc_data, columns=[
        "year", "country", "t_stat", "t_p", "diff_mean", "diff_std"
    ])#.sort_values("diff_mean")
    return df_1luc_change_tt_test, df_1luc_npp_change_tt_test

In [ ]:
df_crop_change_tt_test, df_crop_npp_change_tt_test = ttest_1luc(df_matched_m_all, df_diff_m_all, luc="crop")
df_grass_change_tt_test, df_grass_npp_change_tt_test = ttest_1luc(df_matched_m_all, df_diff_m_all, luc="grass")

# combine ttest

In [ ]:
tt_test_results = [df_crop_change_tt_test, df_crop_npp_change_tt_test, df_grass_change_tt_test, df_grass_npp_change_tt_test, ]
tt_test_col = ["crop_change", "crop_npp_change", "grass_change", "grass_npp_change", ]

In [7]:
def combine_ttest_1lucs(tt_test_results, tt_test_col):
    tt_test_reclass_lst = []
    for tt_test_df, col_ in zip(tt_test_results, tt_test_col):
        tt_test_reclass = tt_test_df\
            .assign(sig=lambda _df: (_df["t_p"]<0.05).astype(int))\
            .assign(increase=lambda _df: (_df["t_stat"]>0).astype(int)* 2 - 1)\
            .assign(type_=lambda _df: _df["sig"] * _df["increase"])\
            .rename(columns={"type_": f"{col_}"})\
            .set_index(["year", "country"])[[f"{col_}"]]
        
        tt_test_reclass_lst.append(tt_test_reclass)
    df_ttest_merge = pd.DataFrame(index=tt_test_reclass.index).copy()
    for tt_test_reclass in tt_test_reclass_lst:
        df_ttest_merge = df_ttest_merge.join(tt_test_reclass, how="outer")
    df_ttest_merge.fillna(0)
    return df_ttest_merge

In [15]:
df_ttest_merge = combine_ttest_1lucs(tt_test_results, tt_test_col)
df_ttest_merge.to_csv(path_data / f"ring/ttest/ring_{ring_radius}km.csv")

In [ ]:
for ring_radius in range(5, 26, 5):
    if (path_data / f"ring/ttest/ring_{ring_radius}km.csv").exists():
        continue
    df_matched_m_all, df_diff_m_all = load_df_diff_matched(ring_radius)
    df_diff_m_all.loc[:, "country"] = df_matched_m_all["country"]
    
    df_crop_change_tt_test, df_crop_npp_change_tt_test = ttest_1luc(df_matched_m_all, df_diff_m_all, luc="crop")
    df_grass_change_tt_test, df_grass_npp_change_tt_test = ttest_1luc(df_matched_m_all, df_diff_m_all, luc="grass")
    
    tt_test_results = [df_crop_change_tt_test, df_crop_npp_change_tt_test, df_grass_change_tt_test, df_grass_npp_change_tt_test, ]
    tt_test_col = ["crop_change", "crop_npp_change", "grass_change", "grass_npp_change", ]
    
    df_ttest_merge = combine_ttest_1lucs(tt_test_results, tt_test_col)
    df_ttest_merge.to_csv(path_data / f"ring/ttest/ring_{ring_radius}km.csv")

In [17]:
# df_ttest_merge

In [ ]:
def ttest_2_ttest_class(df_ttest_merge):
    df_ttest_class_result = df_ttest_merge.map(lambda x: x<0)
    ttest_cols = ["CS", "CD", "GS", "GD"]

    df_ttest_class_result.columns = ttest_cols
    def extract_type(type_lst):
        len_ = len(type_lst)
        match len_:
            case 0:
                return "NA"
            case 1:
                return type_lst[0]
            case _:
                return "MA"

    df_ttest_class_result.loc[:, "class_"] = df_ttest_class_result.apply(lambda x: extract_type(np.array(ttest_cols)[x]), axis=1)
    df_ttest_class_result = df_ttest_class_result.reset_index()
    
    df_ttest_class_result = df_ttest_class_result.query("country != 'China'")
    return df_ttest_class_result

In [ ]:
df_ttest_class_result = df_ttest_merge.map(lambda x: x<0)
ttest_cols = ["CS", "CD", "GS", "GD"]

year  country                 
2002  Algeria                          [FD, GS]
      Angola                                 []
      Burundi                                []
      Central African Republic               []
      Cte d'Ivoire                           []
                                      ...      
2023  Syria                               [CPD]
      Turkey                                 []
      Ukraine                         [CDS, FS]
      Venezuela                       [CDS, FS]
      Yemen                       [CPD, GS, GD]
Length: 508, dtype: object

In [ ]:
ring_radius_s = np.arange(5, 26, 5)
lst_df_ttest_merge = [pd.read_csv(path_data / f"ring/ttest/ring_{ring_radius}km.csv", index_col=[0, 1]) for ring_radius in ring_radius_s]
df_ttest_merge = pd.concat(lst_df_ttest_merge, keys=ring_radius_s, axis=1)
df_ttest_merge = df_ttest_merge.reorder_levels([1, 0], axis=1)

# df_ttest_merge.loc[2023, "crop_change"]

In [14]:
def find_first_nonneg_prev_col(row):
    first_nonneg = (row >= 0).idxmax()
    
    if row[first_nonneg] < 0:
        return row.index[-1]
    if first_nonneg == row.index[0]:
        return 0
    pos = list(row.index).index(first_nonneg)
    return row.index[pos-1]

In [15]:
dic_region = {
    "Northern Africa and Western Asia": "N. Africa W. Asia",
    "Sub-Saharan Africa": "Sub-Saharan Africa",
    "Central Asia and Russian Federation": "C. Asia",
    "Eastern Asia": "E. Asia",
    "Southern Asia": "S. Asia",
    "Southeastern Asia": "S. Asia",
    "Northern America": "N. America",
    "Latin America and the Caribbean": "Latin America",
    "Western Europe": "W. Europe",
    "Eastern and South-Eastern Europe": "E. Europe",
    "Oceania and Australia": "Oceania",
}
gdf_countries = gdf_world.reset_index(drop=True).reset_index().rename(columns={"index": "idx"})
gdf_countries["regi_short"] = gdf_countries["regi_pnas"].map(dic_region)
country_to_region = gdf_countries.set_index("name_long")["regi_short"].to_dict()

In [16]:
regions = ['C. Asia', 'E. Asia', 'S. Asia', 'N. Africa W. Asia', 'Sub-Saharan Africa', 'N. America', 'Latin America', 'Oceania', 'E. Europe', 'W. Europe']
regions_to_color = dict(zip(regions, plt.get_cmap("tab20")(np.linspace(0, 1, len(regions)))))

In [17]:
df_influence_distance = pd.concat([
    df_ttest_merge.loc[:, "crop_change"].apply(find_first_nonneg_prev_col, axis=1).rename("crop_change"),
    df_ttest_merge.loc[:, "crop_npp_change"].apply(find_first_nonneg_prev_col, axis=1).rename("crop_npp_change"),
    df_ttest_merge.loc[:, "grass_change"].apply(find_first_nonneg_prev_col, axis=1).rename("grass_change"),
    df_ttest_merge.loc[:, "grass_npp_change"].apply(find_first_nonneg_prev_col, axis=1).rename("grass_npp_change"),
], axis=1)


In [18]:
df_influence_distance.to_csv(path_data / f"ring/ttest/influence_distance.csv")